In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

## make the dataset zipcode oriented

### load data

In [21]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/ds340/nypd complaint data/1. adding zipcode to initial data/nypd_complaint_cleaned_zipcode.csv')

data.head()

,Unnamed: 0,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,CMPLNT_FR_YEAR,CMPLNT_FR_MONTH,CMPLNT_FR_DAY,geometry,index_right,modzcta,label,zcta,pop_est,zipcode
529869,529834,238514267,01/01/2022,13:35:00,01/01/2022,13:40:00,73.0,01/01/2022,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2,1010953.0,180954.0,45-64,BLACK,M,40.663314,-73.903748,"(40.663314, -73.903748)",PATROL BORO BKLYN NORTH,65+,BLACK,F,2022,1,1,POINT (-73.903748 40.663314),99.0,11212.0,11212,11212,76527.0,11212
529870,529835,238526741,01/01/2022,01:30:00,01/01/2022,13:00:00,103.0,01/01/2022,351,CRIMINAL MISCHIEF & RELATED OF,254.0,"MISCHIEF, CRIMINAL 4, OF MOTOR",COMPLETED,MISDEMEANOR,QUEENS,NaN,STREET,N.Y. POLICE DEPT,0,1043847.0,198594.0,NaN,NaN,NaN,40.711571,-73.785029,"(40.71157064, -73.78502875)",PATROL BORO QUEENS SOUTH,25-44,ASIAN / PACIFIC ISLANDER,M,2022,1,1,POINT (-73.78502875 40.71157064),171.0,11432.0,11432,11432,62053.0,11432
529871,529836,238492710,01/01/2022,00:22:00,01/01/2022,00:25:00,42.0,01/01/2022,118,DANGEROUS WEAPONS,793.0,WEAPONS POSSESSION 3,COMPLETED,FELONY,BRONX,NaN,STREET,N.Y. POLICE DEPT,0,1010441.0,244053.0,NaN,NaN,NaN,40.836506,-73.905350,"(40.83650554, -73.90534959)",PATROL BORO BRONX,UNKNOWN,UNKNOWN,E,2022,1,1,POINT (-73.90534959 40.83650554),60.0,10456.0,10456,10456,94218.0,10456
529872,529837,238516862,01/01/2022,01:52:00,01/01/2022,01:55:00,46.0,01/01/2022,121,CRIMINAL MISCHIEF & RELATED OF,269.0,"MISCHIEF,CRIMINAL, UNCL 2ND",COMPLETED,FELONY,BRONX,INSIDE,OTHER,N.Y. POLICE DEPT,0,1011625.0,251038.0,UNKNOWN,BLACK HISPANIC,M,40.855675,-73.901040,"(40.855675, -73.90104)",PATROL BORO BRONX,UNKNOWN,UNKNOWN,D,2022,1,1,POINT (-73.90104 40.855675),65.0,10457.0,10457,10457,74554.0,10457
529873,529838,238524996,01/01/2022,20:00:00,01/01/2022,20:20:00,113.0,01/01/2022,106,FELONY ASSAULT,105.0,STRANGULATION 1ST,COMPLETED,FELONY,QUEENS,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0,1045309.0,186098.0,25-44,BLACK,M,40.677263,-73.779866,"(40.677263, -73.779866)",PATROL BORO QUEENS SOUTH,25-44,BLACK,F,2022,1,1,POINT (-73.779866 40.677263),173.0,11434.0,11434,"11430, 11434",65989.0,11434


### preprocess

In [4]:
data['zipcode'] = data['zipcode'].astype(str).str.replace('.0', '')

### aggregate the data by 'zipcode'

In [19]:
aggregated_data = data.groupby('zipcode').agg(
    num_felony=pd.NamedAgg(column='LAW_CAT_CD', aggfunc=lambda x: (x == 'FELONY').sum()),
    num_violation=pd.NamedAgg(column='LAW_CAT_CD', aggfunc=lambda x: (x == 'VIOLATION').sum()),
    num_misdemeanor=pd.NamedAgg(column='LAW_CAT_CD', aggfunc=lambda x: (x == 'MISDEMEANOR').sum()),
    most_freq_ofns_desc=pd.NamedAgg(column='OFNS_DESC', aggfunc=lambda x: x.mode()[0]),
    most_freq_loc_occur = pd.NamedAgg(column='LOC_OF_OCCUR_DESC', aggfunc=lambda x: x.mode()[0]),
    most_freq_prem_type=pd.NamedAgg(column='PREM_TYP_DESC', aggfunc=lambda x: x.mode()[0]),
    most_freq_susp_age_group=pd.NamedAgg(column='SUSP_AGE_GROUP', aggfunc=lambda x: x.mode()[0]),
    most_freq_susp_race=pd.NamedAgg(column='SUSP_RACE', aggfunc=lambda x: x.mode()[0]),
    most_freq_susp_sex=pd.NamedAgg(column='SUSP_SEX', aggfunc=lambda x: x.mode()[0]),
    most_freq_vic_age_group=pd.NamedAgg(column='VIC_AGE_GROUP', aggfunc=lambda x: x.mode()[0]),
    most_freq_vic_race=pd.NamedAgg(column='VIC_RACE', aggfunc=lambda x: x.mode()[0]),
    most_freq_vic_sex=pd.NamedAgg(column='VIC_SEX', aggfunc=lambda x: x.mode()[0]),
    pop_est=pd.NamedAgg(column='pop_est', aggfunc='first')
)

aggregated_data['num_total_crimes'] = data.groupby('zipcode')['CMPLNT_NUM'].count()
aggregated_data.reset_index(inplace=True)

aggregated_data = aggregated_data.astype({
    'zipcode': str,
    'num_felony': int,
    'num_violation': int,
    'num_misdemeanor': int,
    'most_freq_ofns_desc': str,
    'most_freq_prem_type': str,
    'num_total_crimes': int
})

aggregated_data.head()

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [20]:
aggregated_data

,zipcode,num_felony,num_violation,num_misdemeanor,most_freq_ofns_desc,most_freq_loc_occur,most_freq_prem_type,most_freq_susp_age_group,most_freq_susp_race,most_freq_susp_sex,most_freq_vic_age_group,most_freq_vic_race,most_freq_vic_sex,pop_est,num_total_crimes
0,10001,2918,734,3677,PETIT LARCENY,INSIDE,STREET,UNKNOWN,BLACK,M,UNKNOWN,UNKNOWN,D,23072.0,7329
1,10002,2278,1024,4381,PETIT LARCENY,INSIDE,STREET,UNKNOWN,UNKNOWN,M,UNKNOWN,UNKNOWN,M,74993.0,7683
2,10003,1865,453,3066,PETIT LARCENY,INSIDE,STREET,UNKNOWN,UNKNOWN,M,UNKNOWN,UNKNOWN,D,54682.0,5384
3,10004,143,70,229,PETIT LARCENY,INSIDE,STREET,UNKNOWN,UNKNOWN,M,UNKNOWN,UNKNOWN,M,3028.0,442
4,10005,219,76,963,PETIT LARCENY,INSIDE,CHAIN STORE,UNKNOWN,BLACK,M,UNKNOWN,UNKNOWN,D,8831.0,1258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,11693,306,189,495,HARRASSMENT 2,INSIDE,STREET,UNKNOWN,BLACK,M,25-44,BLACK,F,12919.0,990
175,11694,266,210,577,PETIT LARCENY,INSIDE,RESIDENCE - APT. HOUSE,UNKNOWN,BLACK,M,UNKNOWN,WHITE,M,21354.0,1053
176,11697,16,6,16,GRAND LARCENY,INSIDE,RESIDENCE-HOUSE,UNKNOWN,WHITE,M,45-64,WHITE,F,3540.0,38
177,99999,493,216,747,PETIT LARCENY,INSIDE,STREET,UNKNOWN,UNKNOWN,M,25-44,UNKNOWN,M,0.0,1456


In [17]:
aggregated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   zipcode                   179 non-null    object 
 1   num_felony                179 non-null    int64  
 2   num_violation             179 non-null    int64  
 3   num_misdemeanor           179 non-null    int64  
 4   most_freq_ofns_desc       179 non-null    object 
 5   most_freq_loc_occur       179 non-null    object 
 6   most_freq_prem_type       179 non-null    object 
 7   most_freq_susp_age_group  179 non-null    object 
 8   most_freq_susp_race       179 non-null    object 
 9   most_freq_susp_sex        179 non-null    object 
 10  most_freq_vic_age_group   179 non-null    object 
 11  most_freq_vic_race        179 non-null    object 
 12  most_freq_vic_sex         179 non-null    object 
 13  pop_est                   178 non-null    float64
 14  num_total_

## export the data

In [22]:
aggregated_data.to_csv('/content/drive/MyDrive/ds340/nypd complaint data/2. covert to zipcode oriented/zipcode_oriented.csv')